In [4]:
import sys
sys.path.append("..")

from misc import load, load_class_labels
from data_loader import data_loader;
import os

dataset_root = "../dataset/"
handcrafted_path = dataset_root + 'handcrafted/'

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')
test_info = load(dataset_root + 'labels/test.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values
y_val = val_labels

class_labels = load_class_labels(dataset_root + 'classes.txt')

dl = data_loader(train_info, train_dir, 400, (224, 224));
dl_val = data_loader(val_info, train_dir, 400, (224, 224));


## Classification using handcrafted features

In [5]:
import numpy as np;
from handcrafted_extraction import load_all_features, load_features
frac_of_train = 1

sift_info = {
    'max_number': 200,
    'num_of_sample_kmeans': 1,
    'voc_size': 300,
    'max_iter': 500
}

gabor_obj = {
    'angles': np.arange(0, np.pi, np.pi/4),
    'lambdas': np.arange(0, 1, 0.2),
    'gammas': [0.5]
}

lbp_info = [1, 2];

sampling = train_info.sample(frac=frac_of_train)
train_image_names = sampling['filename'].values
train_labels = sampling['label'].values

In [6]:
bow_names = os.listdir(handcrafted_path + 'bow_features/')
names = []
labels = []
for name in bow_names:
    index = np.where(train_image_names == name[:-4])
    if len(train_image_names[index]) == 1:
        names.append(train_image_names[index][0])
        labels.append(train_labels[index][0])
train_image_names = names
train_labels = labels

KeyboardInterrupt: 

In [ ]:
color = True
gabor = True
bow = True 
lbp = True

train_features, kmeans = load_all_features(handcrafted_path, train_image_names, load_color=color, load_gabor=gabor, load_bow=bow, load_lbp=lbp) 
val_features, _ = load_all_features(handcrafted_path, val_image_names, load_color=color, load_gabor=gabor, load_bow=bow, load_lbp=lbp)
test_features, _ = load_all_features(handcrafted_path, test_image_names, load_color=color, load_gabor=gabor, load_bow=bow, load_lbp=lbp)

100%|██████████| 11994/11994 [00:00<00:00, 35107.08it/s]


## Standard classification

Common code for features extraction from validation set

In [ ]:
train_and_val_features = train_features.copy();
train_and_val_features = np.concatenate((train_and_val_features, val_features))

all_labels = train_labels.copy()
all_labels = np.concatenate((all_labels, val_labels))

In [ ]:
from sklearn.preprocessing import normalize

normalized_features = normalize(train_and_val_features, axis=1)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

n_neighbors = 15

knn = KNN(n_neighbors)
knn.fit(train_and_val_features, all_labels)

knn_norm = KNN(n_neighbors)
knn_norm.fit(normalized_features, all_labels)

KNeighborsClassifier(n_neighbors=15)

In [ ]:
print(knn.score(test_features, test_labels))
print(knn_norm.score(test_features, test_labels))

0.018842754710688678
0.020343505085876273


### SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_validate, GridSearchCV

parameters = {
    'kernel': ['linear', 'rbf'],
    'C': [1, 10]
}

model = svm.SVC()

Parameter tuning on subset of train (30%)

In [ ]:
clf = GridSearchCV(model, parameters, n_jobs = 8, verbose=3)
#clf.fit(normalized_features, all_labels)

In [ ]:
#print(clf.best_params_)

Using best parameter found (C=1, kernel='rbf')

In [ ]:
model = svm.SVC(C=1, kernel='rbf')
model.fit(normalized_features, all_labels)

In [ ]:
import pickle

with open('./models/svm.model', 'wb') as out:
    pickle.dump(clf, out)

Evaluation

In [ ]:
clf.score(test_features, test_labels)

## Neural network classification

In [ ]:
from tensorflow import keras

model = keras.Sequential([
    keras.Input(shape=train_and_val_features.shape[1]),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(251, activation='softmax')
])

opt = keras.optimizers.Adam(learning_rate=0.02)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [ ]:
import pandas as pd
epoch_num = 10;

labels = keras.utils.to_categorical(all_labels.tolist(), num_classes=251)

for epoch in range(epoch_num):
    model.fit(train_and_val_features, labels, batch_size=100, shuffle=True, validation_split=0.1);

In [ ]:
val_labels = keras.utils.to_categorical(test_labels.tolist(), num_classes=251);
res = model.evaluate(test_features, val_labels)